In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

In [2]:
brakepads = pd.read_json("data.json")
print(brakepads.shape)
brakepads.dropna(subset=['classOfPart'], how='any', inplace=True)
print(brakepads.shape)
brakepads.dropna(thresh=4, inplace=True)
print(brakepads.shape)
brakepads[['classOfPart']].astype('category')
print(brakepads.shape)

(4001, 5)
(1440, 5)
(1275, 5)
(1275, 5)


In [3]:
brakepads.isnull().sum()

partNumber       0
classOfPart      0
length         232
width          768
thickness        0
dtype: int64

In [4]:
brakepads['length'].dtype


dtype('O')

In [5]:
brakepads['length']=brakepads['length'].astype('float')

In [6]:
brakepads['length'] = (brakepads['length'].fillna(brakepads.groupby('classOfPart')['length'].transform('mean')).astype(float))

In [7]:
brakepads['width']=brakepads['width'].astype('float')
brakepads['width'] = (brakepads['width'].fillna(brakepads.groupby('classOfPart')['width'].transform('mean')).astype(float))

In [8]:
brakepads.isnull().sum()

partNumber     0
classOfPart    0
length         0
width          0
thickness      0
dtype: int64

### Splitting of test and training data

In [9]:
from sklearn.model_selection import train_test_split
X = brakepads[['length', 'thickness', 'width']]
y = brakepads[['classOfPart']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Random forest classifier

In [10]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=35, max_depth=1, random_state=42)
clf.fit(X_train, y_train) 
clf.score(X_test, y_test)

<ipython-input-10-d9fb6f035e6d>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)


0.8784313725490196

## Logistic Regression

In [11]:
from sklearn.linear_model import LogisticRegression

In [12]:
lr = LogisticRegression()

In [13]:
lr.fit(X_train,y_train)

/usr/local/Cellar/jupyterlab/3.0.12/libexec/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LogisticRegression()

In [14]:
pred=lr.predict(X_test)

In [15]:
from sklearn.metrics import accuracy_score

In [16]:
print(accuracy_score(y_test,pred))

0.7764705882352941


## KNN Classifier

In [17]:
from sklearn.neighbors import KNeighborsClassifier

In [18]:
knn = KNeighborsClassifier(n_neighbors=3)

In [19]:
knn.fit(X_train,y_train)

/usr/local/Cellar/jupyterlab/3.0.12/libexec/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=3)

In [20]:
pred1=knn.predict(X_test)
print(accuracy_score(y_test,pred1))

0.9568627450980393


In [21]:
test = pd.read_csv("test.csv")

In [22]:
test.head()

,index,length,thickness,width,classOfPart
0,0,108.700000,14.0,71.728235,Hinterachse
1,1,139.747350,16.0,93.100000,Hinterachse
2,2,107.800000,12.9,77.498078,Hinterachse
3,3,101.067871,15.4,53.800000,Hinterachse
4,4,95.000000,15.8,47.696782,Hinterachse


In [23]:
test.drop(['index'],axis=1,inplace=True)

In [24]:
test.isnull().sum()

length         0
thickness      0
width          0
classOfPart    0
dtype: int64

In [25]:
X1_test = test[['length','thickness','width']]
y1_test = test[['classOfPart']]

In [26]:
y1 = clf.predict(X1_test)

In [27]:
print(accuracy_score(y1_test,y1))

0.7501963864886095


In [28]:
y2 = lr.predict(X1_test)
print(accuracy_score(y1_test,y2))

0.7344854673998429


In [29]:
y3 = knn.predict(X1_test)
print(accuracy_score(y1_test,y3))

0.7179890023566379
